In [ ]:
from bluesky import RunEngine
from bluesky.callbacks import LiveTable
from bluesky.plans import scan, count
from bluesky.plan_stubs import mv, open_run, close_run, create, \
    save, drop, rd, trigger, trigger_and_read, declare_stream
from ophyd.sim import det1, noisy_det, motor1, flyer1, flyer2
from databroker import Broker
from matplotlib.pyplot import ion
from bluesky.utils import install_nb_kicker
from bluesky.callbacks.best_effort import BestEffortCallback
from pprint import pprint
from IPython.display import display
from bluesky.preprocessors import subs_decorator, baseline_decorator, monitor_during_decorator, fly_during_decorator
install_nb_kicker()

ion()
RE = RunEngine()

db = Broker.named('temp')
RE.subscribe(db.insert)
bec = BestEffortCallback()
RE.subscribe(bec)
bec.disable_plots()

## Documents

Um dos principais objetivos do Bluesky é habilitar a coleta de dados e metadados. A meneira que ele faz isso é através dos **Documentos**.

Documentos são basicamente dicionários em Python com um esquema definido, ou seja, é um dicionário organizado de uma maneira específica e documentada. Esses **documentos** são gerados pela RunEngine durante a execução de planos e todos os dados e metadados estão organizados dentro dos **Documentos**.

### Runs e eventos

Planos podem gerar dados e metadados, porém como é possível organizar ou agrupar informações em Eventos e Runs?

In [ ]:
def one_run_one_event():
    yield from open_run()
    yield from trigger_and_read([det1, noisy_det])
    yield from close_run()

In [ ]:
RE(one_run_one_event())

Neste exemplo temos:
 - Uma tabela (uma Run)
 - Uma linha (um evento)
 - Duas colunas (além de seq_num e time), representando os dois detectores

In [ ]:
def one_run_n_events():
    yield from open_run()
    for i in range(4):
        yield from trigger_and_read([det1, noisy_det])
    yield from close_run()

In [ ]:
RE(one_run_n_events())

Neste exemplo temos:
 - Uma tabela (uma Run)
 - Quatro linhas (quatro eventos)
 - Duas colunas (além de seq_num e time), representando os dois detectores

In [ ]:
def multi_run():
    for i in range(2):
        yield from one_run_n_events()

In [ ]:
RE(multi_run())

Neste exemplo temos:
 - Duas tabelas (duas Runs)
 - Quatro linhas (quatro eventos por Run)
 - Duas colunas (além de seq_num e time), representando os dois detectores

   
Além disso, agora o retorno da função é uma tupla com multiplos uids, cada um relacionado com uma Run

A definição de uma Run é flexível, pois tudo depende de como você gostaria de organizar seus dados, seja para coleta ou análise.

#### Streams

No bluesky, as streams são fluxos de dados gerados durante uma execução de um plano. De maneira geral, cada stream contém um conjunto de dados e metadados associados a fases específicas de um experimento, por exemplo. 

Os fluxos de dados são organizados nos __event documents__ enviados pela RunEngine durante a execução de um plano

### Documentos de uma Run

#### O Start Document
O Start document contém informações de metadados no início da run:
- time - Tempo de início do plano
- plan_name - Nome do plano
- uid - ID único de identificação da Run
- scan_id - id "mais amigável", porém não é necessáriamente único
- metadados

In [ ]:
uid, = RE(scan([det1, noisy_det], motor1, 0, 10, 6))

In [ ]:
last_run = db[uid]

In [ ]:
last_run.start

#### O Event Document
Vão guardar informações de um ou mais eventos:

In [ ]:
for event in last_run.events():
    display(event)
    break

In [ ]:
pprint(event.to_name_dict_pair())

#### O Stop Document

In [ ]:
last_run.stop

#### Event Descriptor

É um documento que descreve a estrutura dos dados colatados. Servem para organizar os dados e facilitar a interpretação de informações.

##### Data Key
Metadados de cada __data key__

In [ ]:
last_run.descriptors[0]['data_keys']

##### Object Keys
Object keys fornecem informações de como relacionar cada dispositivo com suas data keys. É útil quando uim dispositivo possui múltiplos data keys. Como no caso do **motor1**

In [ ]:
last_run.descriptors[0]['object_keys']

##### Configuration
O configuration permite acessa informaçõe de configuração dos dispotivos, são informações que tipicamente não mudam entre Runs

In [ ]:
last_run.descriptors[0]['configuration']

### Metadados

Metadados são completamente personalizáveis, dado que sua classificação como dado/metadado é sensível ao contexto da aplicação.

Por exemplo, dados que precisamos saber apenas antes do experimento como:
- Nome do usuário
- Parâmaetros da amostra
- Informações da medida

Todas essas informações podem estar descritas no **Start Document**. Existem algumas maneiras de adicionar os metadados:

In [ ]:
uid, = RE(count([det1], num=10), sample_name='Agora vai', bl_operator='Alguém', minhas_informacoes='Só um teste...')

In [ ]:
last_run_with_metadata = db[uid]

In [ ]:
last_run_with_metadata.start

Planos com múltiplas Runs também vão receber os mesmos metadados:

In [ ]:
uid_run1, uid_run2, = RE(multi_run(), sample_name='Agora vai', bl_operator='Alguém', minhas_informacoes='Só um teste...')

In [ ]:
run1 = db[uid_run1]
run2 = db[uid_run2]

In [ ]:
run1.start

In [ ]:
run2.start

Também é possível definir metadados específicos para cada plano, utilizando o parâmetro **md**:

In [ ]:
def my_multi_run_plan():
    yield from scan([det1], motor1, 0, 5, 5, md={'idea': 'find the good region', 'ref': 'Ti'})
    yield from count([det1], num=10, md={'purpose': 'stab'})

In [ ]:
uid1, uid2, = RE(my_multi_run_plan(), RE_metadata={'operator': 'Alguém'}, more_meta={'meta': 'data'})

In [ ]:
multi1 = db[uid1]
multi2 = db[uid2]

In [ ]:
multi1.start

In [ ]:
multi2.start

#### Adicionando metadados para uso repetitivo

In [ ]:
RE.md

In [ ]:
RE.md['proposal'] = 20231320

In [ ]:
RE.md

In [ ]:
uid, = RE(count([det1], num=10))

In [ ]:
my_run = db[uid]

In [ ]:
my_run.start

#### Configurar validações de presença de metadados

In [ ]:
def validate_proposal_number(md):
    if 'proposal' not in md:
        raise ValueError('No proposal number was set!')

    if not isinstance(md['proposal'], int):
        raise ValueError('Proposal must be int.')

In [ ]:
RE.md_validator = validate_proposal_number

In [ ]:
del RE.md['proposal']

In [ ]:
RE(count([det1], num=3))

In [ ]:
RE(count([det1]), proposal='opa')

In [ ]:
RE.md['proposal'] = 20231320

### Supplemental Data

O Suplemental Data faz parte dos Blusky Preprocessors, basicamente preprocessors são funções que alteram planos antes de serem executados, permitindo adições de funcionalidades de maneira automática, sem que o usuário precise realizar interferências manuais em cada etapa

In [ ]:
from bluesky.preprocessors import SupplementalData
sd = SupplementalData()
RE.preprocessors.append(sd)

#### Baseline e Monitor

É um conjunto de dispositivos cujas leituras são automaticamente registradas no início e no final de cada experimento. A função baseline serve para garantir a captura de parâmetros críticos (como temperatura, pressão, etc.), permitindo comparar essas variáveis ao longo do tempo e garantir a consistência dos experimentos.

In [ ]:
sd.baseline = [det1, noisy_det, motor1]

In [ ]:
RE(count([det1], num=3))

In [ ]:
RE(multi_run())

In [ ]:
sd.monitors = [noisy_det]

In [ ]:
uid_preprocessor, = RE(count([det1], num=6))

In [ ]:
my_preprocessor_info = db[uid_preprocessor]

In [ ]:
my_preprocessor_info.stream_names

In [ ]:
my_preprocessor_info.table('baseline')

In [ ]:
my_preprocessor_info.table('noisy_det_monitor')

### Callbacks

A RunEngine, ao executar um plano, organiza dados em matadados em documentos. Cada vez que um documento é criado, a RE passa essa lista de documentos para uma lista de funções.

Cada função pode ter sua implementação específica, desde salvamento de dados até plots em tempo real.

In [ ]:
RE(count([det1], num=3), print)

In [ ]:
def my_callback(name, doc):
    print('From my callback:', name)
    print('Docs', doc)

In [ ]:
RE(count([det1], num=3), my_callback)

### Sidequest: Decorators e Wrappers

Em Python, decorators são funções que podem modificar o comportamento de outras funções e métodos de classes, sem necessáriamente modificar o código-fonte delas. Na prática são utilizados para adicionar funcionalidades antes e depois de uma função.

Já um wrapper é uma função interna, como o __wrapper()__ no exemplo abaixo. Ele envolve a função original e permite a adição de comportamentos adicionais ao redor da execução da função decorada. Note que o decorator recebe a função como argumento e retorna uma nova função, isto é, decorator != wrapper!

In [ ]:
def my_decorator(function):
    def wrapper():
        print('Antes')
        function()
        print('Depois')
    return wrapper

In [ ]:
@my_decorator
def my_func():
    print('Olá pessoal')

In [ ]:
my_func()

#### Callbacks

Sabendo o que são decoradores e wrappers, podemos utilizar alguns decoradores disponibilizados pelo Bluesky. Por exemplo, para adicionar callbacks para planos específicos podemos utilizar o subs_decorator 

In [ ]:
@subs_decorator(print)
def custom_plan():
    yield from count([det1], num=10)

In [ ]:
RE(custom_plan())

#### Baseline

In [ ]:
sd.baseline = []
sd.monitor = []

In [ ]:
@baseline_decorator([noisy_det])
def my_simple_plan():
    yield from scan([det1], motor1, 0, 10, 10)

In [ ]:
RE(my_simple_plan())

Exemplo mais complexo

In [ ]:
def my_other_plan():
    @baseline_decorator([noisy_det, motor1])
    def inner():
        yield from count([det1], num=2)
        yield from scan([det1], motor1, 0, 10, 10)
    yield from inner()

In [ ]:
RE(my_other_plan())

#### Monitor

In [ ]:
def my_monitor_plan():
    
    @monitor_during_decorator([det1])
    def inner():
        yield from scan([noisy_det], motor1, 0, 10, 10)
        yield from mv(motor1, 20)
    yield from inner()


In [ ]:
uid, = RE(my_monitor_plan())

In [ ]:
monitor = db[uid]

In [ ]:
monitor.table('det1_monitor')

#### Fly

Executa o __kickoff__ durante uma Run

In [ ]:
@fly_during_decorator([flyer1, flyer2])
def fly_plan_with_scan():
    yield from scan([noisy_det], motor1, 0, 10, 10)
    yield from mv(motor1, 20)


In [ ]:
uid, = RE(fly_plan_with_scan())

In [ ]:
fly_run = db[uid]

In [ ]:
fly_run.table('flyer1')

('resource', {
    'path_semantics': 'posix',
    'resource_kwargs': {'frame_per_point': 1},
    'resource_path': 'det.h5',
    'root': '/tmp/tmpcvxbqctr/',
    'spec': 'AD_HDF5',
    'uid': '9123df61-a09f-49ae-9d23-41d4d6c6d78})
('datum', {
    'datum_id': '9123df61-a09f-49ae-9d23-41d4d6c6d788/0',
    'datum_kwargs': {'point_number': 0},
    'resource': '9123df61-a09f-49ae-9d23-41d4d6c6d788'}
})